In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from hyperopt import fmin, tpe, hp, STATUS_OK
from hyperopt import SparkTrials
import mlflow

In [2]:
import mlflow
import time
mlflow_uri = "http://mlflow-server:8888/"
mlflow.set_tracking_uri(mlflow_uri)
time.sleep(5)
mlflow.set_experiment("sample_ds_training")


2025/06/06 13:19:48 INFO mlflow.tracking.fluent: Experiment with name 'sample_ds_training' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/518888359703187190', creation_time=1749215988207, experiment_id='518888359703187190', last_update_time=1749215988207, lifecycle_stage='active', name='sample_ds_training', tags={}>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sample_ds_training").master("spark://spark-master:7077") \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/06 13:20:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load the iris dataset from scikit-learn
iris = iris = load_iris()
X = iris.data
y = iris.target

In [5]:
def optimize(C):
    clf = SVC(C = C)

    # Use the cross validation object to compare the model's performance
    accuracy = cross_val_score(clf, X, y).mean()

    return {'loss': -accuracy, 'status': STATUS_OK}

In [6]:
search_space = hp.lognormal('C', 0 , 1.0)
algo = tpe.suggest

In [7]:
from hyperopt import SparkTrials
spark_trials = SparkTrials()

Because the requested parallelism was None or a non-positive value, parallelism will be set to (4), which is Spark's default parallelism (4), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


In [8]:
with mlflow.start_run(run_name='hyperopt') as run:
    argmin = fmin(
    fn= optimize,
    space=search_space,
    algo=algo,
    max_evals = 3,
    trials=spark_trials)

2025/06/06 13:20:16 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

 33%|███▎      | 1/3 [00:03<00:06,  3.01s/trial, best loss: -0.9733333333333334]

100%|██████████| 3/3 [00:04<00:00,  1.34s/trial, best loss: -0.9800000000000001]


Total Trials: 3: 3 succeeded, 0 failed, 0 cancelled.


🏃 View run hyperopt at: http://mlflow-server:8888/#/experiments/518888359703187190/runs/d98ffeebe2024d09bde560048bf8d165
🧪 View experiment at: http://mlflow-server:8888/#/experiments/518888359703187190


In [9]:
# Print the best value found for C
print("Best value found: ", argmin)

Best value found:  {'C': 1.1917966578000898}


In [10]:
spark.stop()